In [8]:
import numpy as np
import pandas as pd
import requests
from datascience import *

Goal: Identify or visualize the impact of COVID on emissions (visualizations of time series data before and when COVID began)

Learning Objectives:
1. How were different sectors impacted by COVID-19?
2. What was the overall effect of COVID-19 on GHG Emissions across all Countries?
3. Can we observe when COVID directly impacted emissions in America?

## Part One: Comparing Yearly Aggregated Data of Total GHG Emissions 

In [9]:
#dataset of yearly GHG emissions across all countries
sectors_emissions = requests.get("https://api.climatetrace.org/emissions_by_sector_timeseries?interval=year&since=2000&to=2021").json()
sectors_emissions = pd.json_normalize(sectors_emissions, record_path = ["emissions"], meta = ["name"]).rename(columns ={"start": "Start Date", "end": "End Date", "value": "GHG Emissions", "name":"Sector"})
sectors_emissions = Tables.from_df(sectors_emissions)

NameError: name 'Tables' is not defined

In [ ]:
sectors_emissions

In [ ]:
sectors_emissions["Sector"].value_counts()

## Part Two: Monthly Data in 2020-2021 USA

In [ ]:
usa_monthly = requests.get("https://api.climatetrace.org/emissions_by_sector_timeseries?country=USA&interval=month&since=2000&to=2021").json()
usa_monthly = pd.json_normalize(usa_monthly, record_path = ["emissions"], meta = ["name"]).rename(columns ={"start": "Start Date", "end": "End Date", "value": "GHG Emissions", "name":"Sector"})
usa_monthly = Tables.from_df(usa_monthly)

In [ ]:
usa_monthly

## Part Three: Observing Daily Data in January-April 2020 USA

In [ ]:
usa_daily = requests.get("https://api.climatetrace.org/emissions_by_sector_timeseries?country=USA&interval=day&since=2020&to=2021").json()
usa_daily = pd.json_normalize(usa_daily, record_path = ["emissions"], meta = ["name"]).rename(columns ={"start": "Start Date", "end": "End Date", "value": "GHG Emissions", "name":"Sector"})
usa_daily

1. Cleaning Dataset to only the Months of January through April
2. Plotting daily ghg emissions data

## Part Four: Try it for yourself
Have students execute the request call using the ClimateTrace API [https://api.climatetrace.org/] to observe time series of another country

1. emissions_by_subsector_timeseries/{country} and select `Try it out`
2. choose country based on their 3 letter abbreviation below and input into `country` text box
3. Change the interval to ___ and the years to ___ and ___
4. Click on `Execute`
5. Copy the `Request URL` and paste it in the code block below


In [ ]:
countries = requests.get("https://api.climatetrace.org/countries").json()
countries = pd.json_normalize(countries).drop("alpha2", axis=1).rename(columns = {"alpha3":"Abbrev"})

In [ ]:
country = requests.get("....").json()
country = pd.json_normalize(country, record_path = ["emissions"], meta = ["name"]).rename(columns ={"start": "Start Date", "end": "End Date", "value": "GHG Emissions", "name":"Sector"})
country = Tables.from_df(country)
country.head()